In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from scipy import stats
from  scipy.stats import norm,skew
import time

In [2]:
train=pd.read_csv("data/train.csv",parse_dates=[2], low_memory=False)
test=pd.read_csv("data/test.csv",parse_dates=[3],low_memory=False)
store=pd.read_csv("data/store.csv")


In [3]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OrdinalEncoder, OneHotEncoder, LabelEncoder
from sklearn.model_selection import train_test_split
import xgboost as xgb
import lightgbm as lgb
import catboost as ctb

In [4]:
print(train["Date"].min(),train["Date"].max())
train.sort_values(["Date"],inplace=True, kind="mergesort")
train.reset_index(drop=True,inplace=True)

2013-01-01 00:00:00 2015-07-31 00:00:00


In [5]:
test["Open"]=test["Open"].fillna(1)

In [6]:
store.isnull().sum()/store.shape[0]

Store                        0.000000
StoreType                    0.000000
Assortment                   0.000000
CompetitionDistance          0.002691
CompetitionOpenSinceMonth    0.317489
CompetitionOpenSinceYear     0.317489
Promo2                       0.000000
Promo2SinceWeek              0.487892
Promo2SinceYear              0.487892
PromoInterval                0.487892
dtype: float64

In [7]:
store.fillna(0,inplace=True)

In [8]:
train=pd.merge(train,store,on="Store",how="left")
test=pd.merge(test,store,on="Store",how="left")

In [9]:
for df in [train,test]:
    df["year"]=df.Date.dt.year
    df["month"]=df.Date.dt.month
    df["day"]=df.Date.dt.day
    df["day"]=df.Date.dt.day
    assert np.all(df.DayOfWeek-1==df["Date"].dt.dayofweek)
    df["dayofyear"]=df.Date.dt.dayofyear
    df["weekofyear"]=df.Date.dt.weekofyear
    df.drop("Date",axis=1,inplace=True)
    

In [10]:
for df in [train,test]:
    df["CompetitionOpen"]=((df["year"]-df["CompetitionOpenSinceYear"])*12 
                          + (df["month"]-df["CompetitionOpenSinceMonth"]))
    df["CompetitionOpen"]=df["CompetitionOpen"].apply(lambda x : x if x>0 else 0)
    df["PromoOpen"]=((df["year"]-df["Promo2SinceYear"]) * 12
                     + (df["weekofyear"]-df["Promo2SinceWeek"])/4)
    df["PromoOpen"]=df["PromoOpen"].apply(lambda x : x if x>0 else 0)
    

In [11]:
month2str = {1:'Jan', 2:'Feb', 3:'Mar', 4:'Apr', 5:'May', 6:'Jun',
             7:'Jul', 8:'Aug', 9:'Sept', 10:'Oct', 11:'Nov', 12:'Dec'}
def check(row):
    if isinstance(row["PromoInterval"],str) and month2str[row["month"]] in row["PromoInterval"]:
        if (row["year"] > row["Promo2SinceYear"] or
            (row['year'] == row['Promo2SinceYear'] and row['weekofyear'] > row['Promo2SinceWeek'])):
            return 1
    return 0

for df in [train,test]:
    df["IsPromoMonth"]=df.apply(lambda row:check(row), axis=1)

### train은 Open 평균이 의미가 있으나, test는 Open 평균이 모두 같으므로 train의 opne 평균을 test에도 적용한다

In [12]:
# train
groups=train[["Store","Open"]].groupby("Store").mean()
groups.rename(columns={"Open":"shopavgopen"},inplace=True)
train = pd.merge(train, groups, how="left", on="Store")
test = pd.merge(test, groups, how="left", on="Store")

In [13]:
groups=train[["Store","Sales","Customers"]].groupby("Store").sum()
groups["ShopAvgSalePerCustomer"]=groups["Sales"]/groups["Customers"]
del groups["Sales"],groups["Customers"]
train = pd.merge(train, groups, how="left", on="Store")
test = pd.merge(test, groups, how="left", on="Store")

In [14]:
groups=train[["Store","SchoolHoliday"]].groupby("Store").mean()
groups.columns=["ShopAvgSchoolHoliday"]
train = pd.merge(train, groups, how="left", on="Store")
test = pd.merge(test, groups, how="left", on="Store")

In [15]:
train.columns

Index(['Store', 'DayOfWeek', 'Sales', 'Customers', 'Open', 'Promo',
       'StateHoliday', 'SchoolHoliday', 'StoreType', 'Assortment',
       'CompetitionDistance', 'CompetitionOpenSinceMonth',
       'CompetitionOpenSinceYear', 'Promo2', 'Promo2SinceWeek',
       'Promo2SinceYear', 'PromoInterval', 'year', 'month', 'day', 'dayofyear',
       'weekofyear', 'CompetitionOpen', 'PromoOpen', 'IsPromoMonth',
       'shopavgopen', 'ShopAvgSalePerCustomer', 'ShopAvgSchoolHoliday'],
      dtype='object')

In [16]:
groups1=train[["Store","Sales"]].groupby("Store").sum()
groups2=train[train["StateHoliday"]!=0][["Store","Sales"]].groupby("Store").sum()
groups=pd.merge(groups1,groups2,on="Store")
groups["ShopSalesHoliday"]=groups["Sales_y"]/groups["Sales_x"]
del groups["Sales_x"],groups["Sales_y"]
train = pd.merge(train, groups, how="left", on="Store")
test = pd.merge(test, groups, how="left", on="Store")

In [17]:
groups1=train[["Store","Sales"]].groupby("Store").sum()
groups2=train[train["IsPromoMonth"]==1][["Store","Sales"]].groupby("Store").sum()
groups=pd.merge(groups1,groups2,on="Store")
groups["ShopSalesPromo"]=groups["Sales_y"]/groups["Sales_x"]
del groups["Sales_x"],groups["Sales_y"]
train = pd.merge(train, groups, how="left", on="Store")
test = pd.merge(test, groups, how="left", on="Store")

In [18]:
groups1=train[["Store","Sales"]].groupby("Store").sum()
groups2=train[train["DayOfWeek"]==6][["Store","Sales"]].groupby("Store").sum()
groups=pd.merge(groups1,groups2,on="Store")
groups["ShopSalesSaturday"]=groups["Sales_y"]/groups["Sales_x"]
del groups["Sales_x"],groups["Sales_y"]
train = pd.merge(train, groups, how="left", on="Store")
test = pd.merge(test, groups, how="left", on="Store")

In [19]:
assert np.all(train[train.Open==0]["Sales"]==0)
train=train[train["Sales"]!=0]
del train["Open"]
test_close_ind=np.where(test["Open"]==0)[0]
del test["Open"]

In [20]:
for col in ["StateHoliday", "StoreType", "Assortment", "DayOfWeek", "month", "PromoInterval"]:
    for val in train[col].unique():
        new_col_name=col+"_"+str(val)
        train[new_col_name]=(train[col]==val).astype(int)
        test[new_col_name]=(test[col]==val).astype(int)
del train["PromoInterval"],test["PromoInterval"]

In [21]:
for col in ["StateHoliday","StoreType","Assortment"]:
    le=LabelEncoder()
    train[col]=le.fit_transform(train[col])
    test[col]=le.transform(test[col])

In [22]:
train.StateHoliday.unique(),train.StoreType.unique(),train.Assortment.unique()

(array([1, 0, 2, 3], dtype=int64),
 array([1, 0, 2, 3], dtype=int64),
 array([0, 1, 2], dtype=int64))

In [23]:
train.StateHoliday.unique(),train.StoreType.unique(),train.Assortment.unique()

(array([1, 0, 2, 3], dtype=int64),
 array([1, 0, 2, 3], dtype=int64),
 array([0, 1, 2], dtype=int64))

In [24]:
y_train=np.array(train.Sales)
train.drop("Sales",axis=1,inplace=True)
train.drop("Customers",axis=1,inplace=True)
test_id=test["Id"]
test.drop("Id",axis=1,inplace=True)

In [25]:
y_train=np.log1p(y_train)

In [26]:
def rmspe(y_pred,y_true):
    y_pred=y_pred[y_pred!=0]
    y_true=y_true[y_true!=0]
    err=np.sqrt(np.mean((1-y_pred/y_true)**2))
    return err
def rmspe_xgb(y_pred,y_true):
    y_true=y_true.get_label()
    err=rmspe(np.expm1(y_pred),np.expm1(y_true))
    return "rmspe",err

In [27]:
start=time.time()
valid_mask=(train.year==2015) & (train.dayofyear>=171)
train1,y_train1=train[~valid_mask],y_train[~valid_mask]
train2,y_train2=train[valid_mask],y_train[valid_mask]
reg=xgb.XGBRegressor(n_estimators=5000,objective="reg:squarederror",max_depth=10,
                    learning_reate=0.03,colsample_bytree=0.7,subsample=0.9,
                    random_state=0, tree_method="gpu_hist")
reg.fit(train1,y_train1,eval_set=[(train1,y_train1),(train2,y_train2)],
       eval_metric=rmspe_xgb,early_stopping_rounds=100,verbose=100)
best_iteration=reg.best_iteration
print("ellapse",time.time()-start)

[0]	validation_0-rmse:5.79232	validation_1-rmse:5.80251	validation_0-rmspe:0.99686	validation_1-rmspe:0.99691
Multiple eval metrics have been passed: 'validation_1-rmspe' will be used for early stopping.

Will train until validation_1-rmspe hasn't improved in 100 rounds.
[100]	validation_0-rmse:0.08397	validation_1-rmse:0.12345	validation_0-rmspe:0.10053	validation_1-rmspe:0.13250
[200]	validation_0-rmse:0.07086	validation_1-rmse:0.12155	validation_0-rmspe:0.07593	validation_1-rmspe:0.13046
Stopping. Best iteration:
[181]	validation_0-rmse:0.07284	validation_1-rmse:0.12149	validation_0-rmspe:0.08041	validation_1-rmspe:0.13033

ellapse 127.50296521186829


In [28]:
pred=reg.predict(test)
train_pre=reg.predict(train)
print(rmspe(np.expm1(train_pre),np.expm1(y_train)))

0.08347402738950004


In [29]:
pred=np.expm1(reg.predict(test))
pred[test_close_ind]=0
submission=pd.DataFrame({"Id":test_id,"Sales":pred},columns=["Id","Sales"])

In [30]:
from datetime import datetime
now=datetime.now()
#submission.to_csv("data/{0:02d}{1:02d}{2:02d}{3:02d}_xgb_submission.csv".format(now.year,now.month,now.day,now.hour),index=False)

In [31]:
474/3303
0.11943

0.11943

## lgb parameter tuning

In [32]:
train.rename(columns = {"PromoInterval_Jan,Apr,Jul,Oct": "PromoInterval_Jan_Apr_Jul_Oct"}, inplace = True)
train.rename(columns = {"PromoInterval_Feb,May,Aug,Nov": "PromoInterval_Feb_May_Aug_Nov"}, inplace = True)
train.rename(columns = {"PromoInterval_Mar,Jun,Sept,Dec": "PromoInterval_Mar_Jun_Sept_Dec"}, inplace = True)

test.rename(columns = {"PromoInterval_Jan,Apr,Jul,Oct": "PromoInterval_Jan_Apr_Jul_Oct"}, inplace = True)
test.rename(columns = {"PromoInterval_Feb,May,Aug,Nov": "PromoInterval_Feb_May_Aug_Nov"}, inplace = True)
test.rename(columns = {"PromoInterval_Mar,Jun,Sept,Dec": "PromoInterval_Mar_Jun_Sept_Dec"}, inplace = True)

In [40]:
gbm = lgb.LGBMRegressor(n_estimators=5000)
gbm.fit(train, y_train,
        eval_metric=rmspe_xgb)

LGBMRegressor(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
              importance_type='split', learning_rate=0.1, max_depth=-1,
              min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
              n_estimators=5000, n_jobs=-1, num_leaves=31, objective=None,
              random_state=None, reg_alpha=0.0, reg_lambda=0.0, silent=True,
              subsample=1.0, subsample_for_bin=200000, subsample_freq=0)

In [34]:
pred=gbm.predict(test)
train_pre=gbm.predict(train)

In [35]:
print(rmspe(np.expm1(train_pre),np.expm1(y_train)))
pred[test_close_ind]=0
submission=pd.DataFrame({"Id":test_id,"Sales":np.round(np.expm1(pred),3)},columns=["Id","Sales"])

0.10617423655643858


In [36]:
from datetime import datetime
now=datetime.now()
#submission.to_csv("data/{0:02d}{1:02d}{2:02d}{3:02d}_lgbm_submission.csv".format(now.year,now.month,now.day,now.hour),index=False)

In [37]:
0.11371
100/3303

0.030275507114744173